In [ ]:
import numpy as np

from sklearn.utils import shuffle

from sklearn.svm import SVC
from sklearn.metrics import f1_score



import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


Load datasets

In [ ]:
datasets =["bear","camel","cat","cow","crocodile","dog","elephant","flamingo",
           "giraffe","hedgehog","horse","kangaroo","lion","monkey",
           "owl","panda","penguin","pig","raccoon","rhinoceros",
           "sheep","squirrel","tiger","whale","zebra"]
train_X =[]
train_y = []
test_X = []
test_y = []
#valid_X = []
#valid_y = []

label = 0
for dataset in datasets:
  
  data_filepath = "/content/drive/MyDrive/dataset/sketchrnn_{dataset}.npz".format(dataset=dataset)
  file = np.load(data_filepath, encoding='latin1', allow_pickle=True)
  
  #test_X.extend(file["test"])  
  #train_X.extend(file["train"])
  #valid_X.extend(file["valid"])

  test_y.extend([label for i in range(file["test"].shape[0])])
  train_y.extend([label for i in range(file["train"].shape[0])])
  #valid_y.extend([label for i in range(file["valid"].shape[0])])

  label += 1

https://github.com/CMACH508/RPCL-pix2seq/blob/main/sample.py

Tranform the data into 28x28 images

In [ ]:
import numpy as np
! pip install svgwrite
!pip install cairosvg
import svgwrite
import cairosvg
import os



def get_bounds(data):
  # Return bounds of data.
  min_x = 0
  max_x = 0
  min_y = 0
  max_y = 0

  abs_x = 0
  abs_y = 0
  for i in range(len(data)):
      x = float(data[i, 0])
      y = float(data[i, 1])
      abs_x += x
      abs_y += y
      min_x = min(min_x, abs_x)
      min_y = min(min_y, abs_y)
      max_x = max(max_x, abs_x)
      max_y = max(max_y, abs_y)

  return (min_x, max_x, min_y, max_y)



def draw_strokes(data, svg_filename='sample.svg', width=48, margin=1.5, color='black'):
  # convert sequence data to svg format
  min_x, max_x, min_y, max_y = get_bounds(data)
  if max_x - min_x > max_y - min_y:
      norm = max_x - min_x
      border_y = (norm - (max_y - min_y)) * 0.5
      border_x = 0
  else:
      norm = max_y - min_y
      border_x = (norm - (max_x - min_x)) * 0.5
      border_y = 0

  # normalize data
  norm = max(norm, 10e-6)
  scale = (width - 2*margin) / norm
  dx = 0 - min_x + border_x
  dy = 0 - min_y + border_y

  abs_x = (0 + dx) * scale + margin
  abs_y = (0 + dy) * scale + margin

  # start converting
  dwg = svgwrite.Drawing(svg_filename, size=(width,width))
  dwg.add(dwg.rect(insert=(0, 0), size=(width,width),fill='white'))
  lift_pen = 1
  p = "M%s,%s " % (abs_x, abs_y)
  command = "m"
  for i in range(len(data)):
      if (lift_pen == 1):
          command = "m"
      elif (command != "l"):
          command = "l"
      else:
          command = ""
      x = float(data[i,0]) * scale
      y = float(data[i,1]) * scale
      lift_pen = data[i, 2]
      p += command+str(x)+","+str(y)+" "
  the_color = color  # "black"
  stroke_width = 1
  dwg.add(dwg.path(p).stroke(the_color,stroke_width).fill("none"))
  dwg.save()



datasets =["bear","camel","cat","cow","crocodile","dog","elephant","flamingo",
      "giraffe","hedgehog","horse","kangaroo","lion","monkey",
      "owl","panda","penguin","pig","raccoon","rhinoceros",
      "sheep","squirrel","tiger","whale","zebra"]
pixel_file_location = "/content/drive/MyDrive/dataset_pixel/"

for dataset in datasets:
  data_filepath = "/content/drive/MyDrive/dataset/sketchrnn_{dataset}.npz".format(dataset=dataset)
  file = np.load(data_filepath, encoding='latin1', allow_pickle=True)
  # cut into train, test, valid
  for tag in ["train","test","valid"]:
    data = file[tag]
    # create directory
    pixel_file_path = pixel_file_location + "/" + tag + "/" + dataset
    os.makedirs(pixel_file_path)

    # iterate for each stroke
    for i in range(len(data)):
        svg_file_path = pixel_file_path + "/" + str(i) + ".svg"
        png_file_path = pixel_file_path + "/" + str(i) + ".png"

        # draw stroke
        stroke_data = data[i]
        draw_strokes(stroke_data,
                      svg_filename=svg_file_path,
                      width=28)

        # convert svg to png
        cairosvg.svg2png(url=svg_file_path, write_to=png_file_path)
        
        os.remove(svg_file_path)
  print(dataset+' done')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67 kB 3.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 88 kB 5.0 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89668 sha256=e647dc884e0e5240e2619036f668be1ac850dcfbd43d5f98fb977e6b4135ce5d
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi
bear done
camel done


In [ ]:
import torchvision
import torch.utils.data.dataloader as DataLoader
from torchvision import transforms

In [ ]:
transform=transforms.Compose([transforms.Grayscale(),transforms.ToTensor()])

train_X = []
test_X = []
train_y = []
test_y = []


# training set
data_path_train =  "/content/drive/MyDrive/dataset_pixel/" + "train"

i=0
train_dataset = torchvision.datasets.ImageFolder(root=data_path_train,transform=transform)
for image_data in train_dataset:
  if i== 70000:#try less classes
    break
  train_X.append(image_data[0].detach().numpy()[0].reshape(28*28))
  #train_y.append(image_data[1])
  i+=1

# test set
data_path_test =  "/content/drive/MyDrive//dataset_pixel/" + "test"

i=0
test_dataset = torchvision.datasets.ImageFolder(root=data_path_test,transform=transform)
for image_data in test_dataset:
  if i== 25000: #try less classes
    break
  test_X.append(image_data[0].detach().numpy()[0].reshape(28*28))
  #test_y.append(image_data[1])
  i+=1


In [ ]:
train_X = np.array(train_X)
test_X = np.array(test_X)

test_y = []
train_y = []
for label in range(0,10):
  train_y.extend([label for i in range(7000)])
  test_y.extend([label for i in range(2500)])

#train_y.sort()

train_X, train_y = shuffle(train_X, train_y, random_state=0)

MODEL

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
losses = ["hinge","log"]#
for l in losses:
  print(l)
  clf = SGDClassifier(loss=l,shuffle = False)
  
  for X,y in zip(np.array_split(train_X[:], 10),np.array_split(train_y[0:70000], 10)):
    clf.partial_fit(X.reshape(-1,1),y,classes=[0,1,2,3,4,5,6,7,8,9])
 
  y_pred = clf.predict(test_y[0:25000])
  sc = clf.score(X=test_X[0:25000],y=test_y[0:25000])
  f1 = f1_score(test_y, y_pred,average='macro')
  print(f'score: {(sc*100):.2f}%')
  print(f'f1-score: {(f1*100):.2f}%')

hinge


ValueError: ignored

In [ ]:
#crash when use full dataset
kernels = ["poly","rbf","sigmoid"]
for k in kernels:
  print(k)
  #for c in range(1,11):
  svm = SVC(C=c, kernel=k)
  svm.fit(train_X,train_y)
  y_pred = svm.predict(test_y)
  sc = svm.score(X=test_X,y=test_y)
  f1 = f1_score(test_y, y_pred,average='macro')
  print(f'score: {(sc*100):.2f}%')
  print(f'f1-score: {(f1*100):.2f}%')



In [ ]:
from sklearn.decomposition import PCA
n = [256,200,128,100,64,50]
for i in n:
  print(n)
  pca = PCA(n_components=2)
  pca.fit(train_X)
  X = pca.transform(train_X)
  svm = SVC(C=1, kernel='rbf')
  svm.fit(train_X,train_y)
  y_pred = svm.predict(test_y)
  sc = svm.score(X=test_X,y=test_y)
  f1 = f1_score(test_y, y_pred,average='macro')
  print(f'score: {(sc*100):.2f}%')
  print(f'f1-score: {(f1*100):.2f}%')

In [ ]:
model = keras.Sequential()

# model.add(tf.layers.conv1d(filters=48,
#       kernel_size=5,activation=None,
#       strides=1)

# model.add(tf.layers.conv1d(filters=64,
#       kernel_size=3,activation=None,
#       strides=1))

# model.add(tf.layers.conv1d(filters=96,
#       kernel_size=3,activation=None,
#       strides=1))

model.add(tf.keras.layers.LSTM(16,return_sequences=True,input_shape=(None, 3)))
model.add(tf.keras.layers.LSTM(16,return_sequences=True,input_shape=(None, 3)))

# softmax classifier
model.add(tf.keras.layers.Dense(.25))#25 classes
model.add(tf.keras.layers.Activation("softmax"))

In [ ]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])
X_new = list(X_new)
train_y = np.array(train_y)
model.fit(X_new,train_y,batch_size=1)

ValueError: ignored

In [ ]:
y_pred = model.preidct(Xt_new)
f1 = f1_score(test_y, y_pred,average='macro')
print(f'f1-score: {(f1*100):.2f}%')